In [ ]:
import sys

sys.path.append('D:\PythonProjects\\bk-monitor\\venv2\share\scripts')
import hello

In [2]:
new_strategy = {
    "type": "monitor",
    "bk_biz_id": 2,
    "scenario": "os",
    "name": "test_strategy_1",
    "labels": [],
    "is_enabled": True,
    "priority": 100,
    "items": [
        {
            "name": "test_item_1",
            "no_data_config": {
                "continuous": 10,
                "is_enabled": False,
                "agg_dimension": [],
                "level": 2
            },
            "target": [],
            "expression": "a",
            "functions": [],
            "origin_sql": "histogram_quantile(0.99, sum(rate(request_duration_seconds_bucket[5m])) by (le))\r\n",
            "query_configs": [
                {
                    "data_source_label": "prometheus",
                    "data_type_label": "time_series",
                    "promql": "histogram_quantile(0.99, sum(rate(request_duration_seconds_bucket[5m])) by (le))\r\n",
                    "agg_interval": 60,
                    "alias": "a"
                }
            ],
            "algorithms": [
                {
                    "level": 2,
                    "type": "Threshold",
                    "config": [
                        [
                            {
                                "method": "gte",
                                "threshold": "90"
                            }
                        ]
                    ],
                    "unit_prefix": ""
                }
            ]
        }
    ],
    "detects": [
        {
            "level": 2,
            "expression": "",
            "trigger_config": {
                "count": "1",
                "check_window": 5,
                "uptime": {
                    "calendars": [],
                    "time_ranges": [
                        {
                            "start": "00:00",
                            "end": "23:59"
                        }
                    ]
                }
            },
            "recovery_config": {
                "check_window": 5,
                "status_setter": "recovery"
            },
            "connector": "and"
        }
    ],
    "actions": [],
    "metric_type": "time_series"
}

In [3]:
import copy
import random
from mock import MagicMock
from core.drf_resource import api
from bkmonitor.strategy.new_strategy import Strategy
from bkmonitor.models.strategy import StrategyModel, ItemModel, DetectModel, AlgorithmModel
from bkmonitor.management.commands.run_cmd import parse_histogram_quantile_strategy

biz_list = [biz.bk_biz_id for biz in api.cmdb.get_business()]


def create_strategy():
    # 清空数据  
    strategy_ids = StrategyModel.objects.all().values_list("id", flat=True)
    try:
        ItemModel.objects.filter(strategy_id__in=strategy_ids).delete()
        DetectModel.objects.filter(strategy_id__in=strategy_ids).delete()
        AlgorithmModel.objects.filter(strategy_id__in=strategy_ids).delete()
    except Exception as e:
        raise e
    else:
        StrategyModel.objects.all().delete()

    # 创建符合预期的数据
    for i in range(1, 10):
        new_strategy["bk_biz_id"] = random.choice(biz_list)
        new_strategy["name"] = f"test_strategy_{i}"
        new_strategy["items"][0]["name"] = f"test_item_{i}"
        strategy = Strategy(**new_strategy)
        strategy.access_aiops = MagicMock(return_value=True)
        strategy.save()

    # 创建count不等于1的策略
    other_strategy_1 = copy.deepcopy(new_strategy)
    other_strategy_1["detects"][0]["trigger_config"]["count"] = 2
    for i in range(1, 10):
        other_strategy_1["bk_biz_id"] = random.choice(biz_list)
        other_strategy_1["name"] = f"test_other_strategy_1{i}"
        other_strategy_1["items"][0]["name"] = f"test_other_item_1{i}"
        strategy = Strategy(**other_strategy_1)
        strategy.access_aiops = MagicMock(return_value=True)
        strategy.save()

    #创建检测算法不为"Threshold" 的策略
    other_strategy_2 = copy.deepcopy(new_strategy)
    other_strategy_2["items"][0]["algorithms"][0]["type"] = "SimpleRingRatio"
    for i in range(1, 10):
        other_strategy_2["bk_biz_id"] = random.choice(biz_list)
        other_strategy_2["name"] = f"test_other_strategy_2{i}"
        other_strategy_2["items"][0]["name"] = f"test_other_item_2{i}"
        strategy = Strategy(**other_strategy_2)
        strategy.access_aiops = MagicMock(return_value=True)
        strategy.save()

    # 创建promql不包含histogram_quantile 的策略
    other_strategy_3 = copy.deepcopy(new_strategy)
    other_strategy_3["items"][0]["origin_sql"] = ""
    for i in range(1, 10):
        other_strategy_3["bk_biz_id"] = random.choice(biz_list)
        other_strategy_3["name"] = f"test_other_strategy_3{i}"
        other_strategy_3["items"][0]["name"] = f"test_other_item_3{i}"
        strategy = Strategy(**other_strategy_3)
        strategy.access_aiops = MagicMock(return_value=True)
        strategy.save()


create_strategy()

# 获取所有策略ID
strategy_ids = StrategyModel.objects.all().values_list("id", flat=True)
print("总共的策略数量", len(strategy_ids))
print()


def test_parse_histogram_quantile_strategy():
    parse_histogram_quantile_strategy()


test_parse_histogram_quantile_strategy()





总共的策略数量 36

业务id、业务名、策略id、策略名:
39、<Business: 39-[39]审计中心 (业务)>、730、test_strategy_1
41、<Business: 41-[41]qwes (业务)>、731、test_strategy_2
34、<Business: 34-[34]aaaaa (业务)>、732、test_strategy_3
3、<Business: 3-[3]测试业务 (业务)>、733、test_strategy_4
35、<Business: 35-[35]cd-test (业务)>、734、test_strategy_5
4、<Business: 4-[4]DBA (业务)>、735、test_strategy_6
40、<Business: 40-[40]heroge-test (业务)>、736、test_strategy_7
2、<Business: 2-[2]蓝鲸 (业务)>、737、test_strategy_8
4、<Business: 4-[4]DBA (业务)>、738、test_strategy_9
